In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/dataset/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/dataset


In [3]:
df=pd.read_csv('Copy of vehicles.csv') ## Deleted Data file because of lack of space ! To get original data, go to this link : https://www.kaggle.com/austinreese/craigslist-carstrucks-data
df=pd.DataFrame(df)
df.head() ## Data is uploaded on my Google Drive !

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [4]:
## Put price column at the last !
df= df.reindex(columns=['id', 'url', 'region', 'region_url', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long','price'])

# Removing useless features

In [ ]:
df2=df.copy()
df2=df2.drop(columns=['url','region_url','vin','image_url','description','county','state'])
df2.head()

# 2) Checking for NULL Values

In [ ]:
df2.isnull().sum()

In [29]:
## Separating numeric variable and categorical variable
categorical_cols=['region','manufacturer','model','condition','cylinders','fuel','title_status','transmission','drive','size','type','paint_color']
numerical_cols=['year','odometer','lat','long']

In [ ]:
df2.head()

In [31]:
## The categorical data seems to have some order in them. Eg : More cylinders would result in more price. Hence we are using Label Encoder !!
## Else we could have used One-Hot-Encoding

check_imputer=df2.copy()
check_imputer=check_imputer.drop(columns=['id'])
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

## l = encoder.fit_transform(check_imputer[categorical_cols])
## test = l.transform(check_imputer[categorical_cols]) 

## Running a function because of reshaping issues 

# # Encoding categorical features ignoring the NaN values !!
def encode(data):
    nonulls = np.array(data.dropna())
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assigning encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

## loop to label each column
for i in range(len(categorical_cols)):
    encode(check_imputer[categorical_cols[i]])


In [ ]:
check_imputer.head()

In [ ]:
df2.isnull().sum()[numerical_cols]

In [34]:
df2 = dataset = df2.dropna(how='all') ## Dropping any row with all NaN values !!
df2['odometer'].fillna((df2['odometer'].mean()), inplace=True) ## Filling with mean value of odometer
df2 = dataset = df2.dropna(subset = ["lat", "long"], how='any') ## Dropping lat and long NULL values. Doesn't make sense for me to use a regression model to predict missing latitude and longitude for now
## Might implement latitude and longitude prediction model later!
## Dropping year too!


In [ ]:
df2.shape

In [ ]:
#Missing values after filling
df2.isnull().sum()[numerical_cols]

In [ ]:
df2.isnull().sum()[categorical_cols] ## Checking for Null Values in categorical columns

In [40]:
data_cat=df2[categorical_cols]
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import IterativeImputer

encoder=LabelEncoder()
from sklearn.linear_model import BayesianRidge ## Regularized Linear Regression Model !!

# Creating a for loop to iterate through each column in the data and hence fill in NaN values.
## StackOverFlow helped a lot in this !

for columns in categorical_cols:
    encode(data_cat[columns])
    imputer = IterativeImputer(BayesianRidge())  ## On some research, I found out some Iterative Methods for imputing values. Decided to try on Bayesian Ridge. Might as well try on some other Regression model
    impute_data=imputer.fit_transform(data_cat[columns].values.reshape(-1, 1))
    impute_data=impute_data.astype('int64') ## type casting to int64 ... was giving some errors !!
    impute_data = pd.DataFrame(impute_data)
    impute_data =encoder.inverse_transform(impute_data.values.reshape(-1, 1)) ## Reshaping the data
    data_cat[columns]=impute_data
df2[categorical_cols]=data_cat ## Putting all values   in df2! 

In [ ]:
df2.head()

In [42]:
df2.isnull().sum()[categorical_cols]

region          0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
dtype: int64

In [43]:
df2.to_csv('vehiclesData.csv',index=False) ## Saving this Data for furthur prediction model!!

(420331, 18)